In [12]:
import tensorflow.compat.v1 as tf
import numpy as np
tf.compat.v1.disable_eager_execution() # need to disable eager in TF2.x

# tf.Session

[tf.Session](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/Session)

A `Session` object encapsulates the environment in which `Operation` objects are executed, and `Tensor` objects are evaluated. For example:

In [13]:
# Build a graph.
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

# Launch the graph in a session.
sess = tf.compat.v1.Session()

# Evaluate the tensor `c`.
print(sess.run(c)) # prints 30.0

30.0


A session may own resources, such as [`tf.Variable`](https://www.tensorflow.org/api_docs/python/tf/Variable)www.tensorflow.org/api_docs/python/tf/queue/QueueBase), and [`tf.compat.v1.ReaderBase`](https://www.tensorflow.org/api_docs/python/tf/compat/v1/ReaderBase). It is important to release thees when they are no longer required. To do this, either invoke the `tf.Session.close` method on the session, or use the session as a context manager. The following two examples are equivalent:

In [14]:
# Using the `close()` method.
sess = tf.compat.v1.Session()
sess.run(c)
sess.close()

In [15]:
# Using the context manager.
with tf.compat.v1.Session() as sess:
  sess.run(c)
# sess.run(c) => closed session error

## session.as_default()

Returns a context manager that makes this object the default session.

Use with the `with` keyword to specify that calls to [`tf.Operation.run`](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/Operation#run) or [`tf.Tensor.eval`](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/Tensor#eval) should be executed in this sessio

In [16]:
# as_default() to use tensor.eval()
# not close automatically
sess1 = tf.compat.v1.Session()
with sess1.as_default():
  assert tf.compat.v1.get_default_session() is sess1
  print(c.eval())

# Note: The as_default context manager does not close the session when you exit the context, 
# and you must close the session explicitly.
sess1.run(a)
sess1.close()

30.0


In [17]:
# Alternatively, you can use with tf.compat.v1.Session(): 
# to create a session that is automatically closed on exiting the context, 
# including when an uncaught exception is raised.
with tf.compat.v1.Session():
  print(c.eval())

30.0


The [`ConfigProto`](https://www.tensorflow.org/code/tensorflow/core/protobuf/config.proto) protocol buffer exposes various configuration options for a session. For example, to create a session that uses soft constraints for device placement, and log the resulting placement decisions, create a session as follows:

In [18]:
# Launch the graph in a session that allows soft device placement and
# logs the placement decisions.
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(
    allow_soft_placement=True,
    log_device_placement=True))

Device mapping: no known devices.


# tf.Tensor

[tf.Tensor](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/Tensor)

In [19]:
x = tf.constant([[2.0, 3.0]])
y = tf.constant([[3.0, -5.0]])
x, y

(<tf.Tensor 'Const_16:0' shape=(1, 2) dtype=float32>,
 <tf.Tensor 'Const_17:0' shape=(1, 2) dtype=float32>)

## tf.convert_to_tensor

In [20]:
def my_func(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.float32)
  return tf.matmul(arg, arg) + arg

# The following calls are equivalent.
value_1 = my_func(tf.constant([[1.0, 2.0], [3.0, 4.0]]))
value_2 = my_func([[1.0, 2.0], [3.0, 4.0]])
value_3 = my_func(np.array([[1.0, 2.0], [3.0, 4.0]], dtype=np.float32))

In [21]:
sess.run((value_1, value_2, value_3)), value_1, value_2, value_3

((array([[ 8., 12.],
         [18., 26.]], dtype=float32),
  array([[ 8., 12.],
         [18., 26.]], dtype=float32),
  array([[ 8., 12.],
         [18., 26.]], dtype=float32)),
 <tf.Tensor 'add_5:0' shape=(2, 2) dtype=float32>,
 <tf.Tensor 'add_6:0' shape=(2, 2) dtype=float32>,
 <tf.Tensor 'add_7:0' shape=(2, 2) dtype=float32>)

In [22]:
# Launch the graph in a session.
sess2 = tf.compat.v1.Session()
with sess2.as_default():
    # Strip leading and trailing 2 elements
    foo = tf.constant([1,2,3,4,5,6])
    print(foo[2:-2].eval())  # => [3,4]
    
    # Skip every other row and reverse the order of the columns
    foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
    print(foo[::2,::-1].eval())  # => [[3,2,1], [9,8,7]]

    # Use scalar tensors as indices on both dimensions
    print(foo[tf.constant(0), tf.constant(2)].eval())  # => 3

    # Insert another dimension
    foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
    print(foo[tf.newaxis, :, :].eval()) # => [[[1,2,3], [4,5,6], [7,8,9]]]
    print(foo[:, tf.newaxis, :].eval()) # => [[[1,2,3]], [[4,5,6]], [[7,8,9]]]
    print(foo[:, :, tf.newaxis].eval()) # => [[[1],[2],[3]], [[4],[5],[6]], [[7],[8],[9]]]

    # Ellipses (3 equivalent operations)
    foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
    print(foo[tf.newaxis, :, :].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]
    print(foo[tf.newaxis, ...].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]
    print(foo[tf.newaxis].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]

    # Masks
    foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
    print(foo[foo > 2].eval())  # => [3, 4, 5, 6, 7, 8, 9]

[3 4]
[[3 2 1]
 [9 8 7]]
3
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]
[[[1 2 3]]

 [[4 5 6]]

 [[7 8 9]]]
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]

 [[7]
  [8]
  [9]]]
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]
[3 4 5 6 7 8 9]


***

# tf.Variable

[tf.Variable](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/Variable)

When you launch the graph, variables have to be explicitly initialized before you can run Ops that use their value. You can initialize a variable by running its *initializer op*, restoring the variable from a save file, or simply running an `assign` Op that assigns a value to the variable. In fact, the variable *initializer op* is just an `assign` Op that assigns the variable's initial value to the variable itself.

In [23]:
x = tf.constant([[2.0, 3.0]])
y = tf.constant([[3.0, -5.0]])
x, y, x.get_shape()

(<tf.Tensor 'Const_28:0' shape=(1, 2) dtype=float32>,
 <tf.Tensor 'Const_29:0' shape=(1, 2) dtype=float32>,
 TensorShape([1, 2]))

In [24]:
# Create a variable.
w = tf.Variable([[1.0], [1.0]])
#w = tf.Variable(tf.ones([2, 1]))
w.get_shape()

TensorShape([2, 1])

In [25]:
# Use the variable in the graph like any Tensor.
x1 = tf.matmul(w, x)

# The overloaded operators are available too.
z = tf.sigmoid(w + x1)
x1, z

(<tf.Tensor 'MatMul_6:0' shape=(2, 2) dtype=float32>,
 <tf.Tensor 'Sigmoid:0' shape=(2, 2) dtype=float32>)

In [26]:
ass_op = tf.assign(w, tf.ones([2, 1])) # work
# ass_op = w.assign(tf.ones([2, 1])) not work because w is not initialized

In [27]:
sess_v = tf.Session()

In [28]:
sess_v.run(ass_op) # the same as sess_v.run(w.initializer)

array([[1.],
       [1.]], dtype=float32)

In [29]:
sess_v.run((x1, z))

(array([[2., 3.],
        [2., 3.]], dtype=float32),
 array([[0.95257413, 0.98201376],
        [0.95257413, 0.98201376]], dtype=float32))

In [30]:
# Assign a new value to the variable with `assign()` or a related method.
w.assign(tf.add(w, 1.0))

<tf.Variable 'AssignVariableOp_1' shape=(2, 1) dtype=float32>

In [31]:
w.assign_add(tf.constant([[1.0], [1.0]]))

<tf.Variable 'AssignAddVariableOp' shape=(2, 1) dtype=float32>

In [32]:
sess_v.close()

***

In [33]:
# Create a variable.
w = tf.Variable(0.)
# Use the variable in the graph like any Tensor.
op = w.assign_add(1.)
w, op

(<tf.Variable 'Variable_1:0' shape=() dtype=float32>,
 <tf.Variable 'AssignAddVariableOp_1' shape=() dtype=float32>)

In [34]:
sess2 = tf.Session()
sess2.run(w.initializer)

In [35]:
# ...you now can run ops that use the value of 'w'...
ret_op = sess2.run(op)
ret_w = sess2.run(w)
ret_both = sess2.run((op, w))
op, ret_op, w, ret_w, ret_both

(<tf.Variable 'AssignAddVariableOp_1' shape=() dtype=float32>,
 1.0,
 <tf.Variable 'Variable_1:0' shape=() dtype=float32>,
 1.0,
 (2.0, 1.0))

In [36]:
y = x * w
sess2.run((y, x, w)), y, x, w

((array([[4., 6.]], dtype=float32), array([[2., 3.]], dtype=float32), 2.0),
 <tf.Tensor 'mul_2:0' shape=(1, 2) dtype=float32>,
 <tf.Tensor 'Const_28:0' shape=(1, 2) dtype=float32>,
 <tf.Variable 'Variable_1:0' shape=() dtype=float32>)

***

# tf.function

[tf.function](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/function)

```
tf.function(
    func=None, input_signature=None, autograph=True,
    experimental_autograph_options=None, experimental_relax_shapes=False,
    experimental_compile=None
)
```

`function` constructs a callable that executes a TensorFlow graph (`tf.Graph`) created by tracing the TensorFlow operations in `func`. This allows the TensorFlow runtime to apply optimizations and exploit parallelism in the computation defined by `func`.

In [37]:
def f(x, y):
  return tf.reduce_mean(tf.multiply(x ** 2, 3) + y)

In [38]:
g = tf.function(f)
x_1 = tf.constant([[2.0, 3.0]])
y_1 = tf.constant([[3.0, -5.0]])

In [39]:
sess = tf.Session()
# `f` and `g` will return the same value, but `g` will be executed as a TensorFlow graph.
sess.run(g(x_1, y_1)), sess.run(f(x_1, y_1))

(18.5, 18.5)

In [40]:
# Tensors and tf.Variables used by the Python function are captured in the graph.
@tf.function
def h():
  return f(x_1, y_1)

In [41]:
sess.run(h())

18.5

In [42]:
# Data-dependent control flow is also captured in the graph.
# Supported control flow statements include `if`, `for`, `while`, `break`, `continue`, `return`.
@tf.function
def gcond(x):
  if tf.reduce_sum(x) > 0:
    return x * x
  else:
    return -x // 2

In [43]:
sess.run(gcond(x_1)), sess.run(gcond(y_1))

(array([[4., 9.]], dtype=float32), array([[-2.,  2.]], dtype=float32))

In [56]:
# print and TensorFlow side effects are supported, but exercise caution when
# using Python side effects like mutating objects, saving to files, etc.
l = []
v = tf.Variable(0)

@tf.function
def gpy(x):
  for i in x:
    print(i)                              # Works
    tf.compat.v1.assign(v, i)                       # Works
    #tf.compat.v1.py_func(lambda i: l.append(i))(i)  # Works
    #l.append(i)                           # Caution! Doesn't work.

Note that unlike other TensorFlow operations, we don't convert python numerical inputs to tensors. Moreover, a new graph is generated for each distinct python numerical value, for example calling `g(2)` and `g(3)` will generate two new graphs (while only one is generated if you call `g(tf.constant(2))` and `g(tf.constant(3))`). Therefore, python numerical inputs should be restricted to arguments that will have few distinct values, such as hyperparameters like the number of layers in a neural network. This allows TensorFlow to optimize each variant of the neural network.

In [57]:
sess.run(v.initializer)
sess.run(gpy([1, 2]))

1


TypeError: in user code:

    <ipython-input-52-28cc5f64b4ed>:11 gpy  *
        tf.compat.v1.py_func(lambda i: l.append(i))(i)  # Works
    /home/jy/anaconda3/envs/tensor_adv/lib/python3.9/site-packages/tensorflow/python/util/deprecation.py:340 new_func  **
        return func(*args, **kwargs)
    /home/jy/anaconda3/envs/tensor_adv/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)

    TypeError: py_func() missing 2 required positional arguments: 'inp' and 'Tout'


The Python function `func` may reference stateful objects (such as [`tf.Variable`](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/Variable)). These are captured as implicit inputs to the callable returned by `function`. For example:

In [58]:
with tf.compat.v1.Session() as sess1:
    c = tf.Variable(0)
    sess1.run(c.initializer)
    
    @tf.function
    def f1(x):
      c.assign_add(1)
      return x + tf.compat.v1.to_float(c)

    print(c.eval())
    print(f1(1).eval())
    print(c.eval())
    print(f1(1).eval())

0
2.0
1
3.0
